In [1]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.db import DBConnection, execute_query, get_pitcher_data
from config import DBConfig

In [2]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [3]:
pd.set_option('display.max_columns', 150)

In [4]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['game_context', 'pitchers', 'sqlite_sequence', 'pitch_mix_features', 'team_stats', 'pitcher_ids', 'teams', 'games', 'statcast_pitches', 'game_stats', 'pitch_mix', 'team_batting_stats', 'batter_profiles', 'sequence_patterns', 'prediction_features', 'starter_probability']


In [5]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM teams LIMIT 10")
    teams = cursor.fetchall()
    print([team[1] for team in teams])

['LAD', 'NYY', 'BOS', 'CHC', 'SFG', 'NYM', 'HOU', 'ATL', 'PHI', 'OAK']


In [6]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM statcast_pitches")
    game_context = cursor.fetchall()
    
df = pd.DataFrame(game_context)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,1,434378,746578,FF,2024-09-28,93.6,-1.65,7.08,2302.0,-0.57,1.53,-0.26,4.32,666310,L,strikeout,swinging_strike,11.0,1,2,57,4,6,Bot,2,647304.0,NaN,NaN
1,2,434378,746578,FF,2024-09-28,92.9,-1.65,7.10,2314.0,-0.88,1.30,-0.18,3.89,666310,L,None,swinging_strike,11.0,1,1,57,3,6,Bot,2,647304.0,NaN,NaN
2,3,434378,746578,CH,2024-09-28,84.8,-1.94,6.86,1899.0,-1.21,0.83,-0.28,2.18,666310,L,None,foul,4.0,1,0,57,2,6,Bot,2,647304.0,NaN,NaN
3,4,434378,746578,CU,2024-09-28,76.6,-1.90,6.87,2624.0,0.41,-1.04,0.46,0.65,666310,L,None,blocked_ball,14.0,0,0,57,1,6,Bot,2,647304.0,NaN,NaN
4,5,434378,746578,CU,2024-09-28,78.0,-1.93,6.95,2739.0,0.61,-1.19,-0.76,1.06,665926,L,strikeout,swinging_strike,13.0,0,2,56,5,6,Bot,1,647304.0,NaN,NaN
